In [ ]:
# Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# aqua-rat json 파일 기준으로 작성된 코드입니다.
import pandas as pd
from tqdm import tqdm
import json
import time
import os
import sys
import urllib.request
import requests

In [ ]:
# 작업할 Google drive 경로 입력
PATH = '/content/drive/MyDrive/Translate_Google'

os.chdir(PATH)
print(os.getcwd())
!ls

/content/drive/MyDrive/Translate_Google
 0_translate_free.csv		    asdiv-a
 11_translate.csv		   'Keyword-sentence Prompts_Korean.gsheet'
 1819_translate_free.csv	    mwp-korean-34ea94931579.json
 1819_translate_free.gsheet	    omega-signifier-332318-8141534bafa6.json
 24368_translate.csv		    train_태우.json
 6532_translate_free.csv	    Translate_Google_free.ipynb
 ap04-123456789-a497a5dc89fd.json   Translate_Google.ipynb




---



Google Translation API  
https://cloud.google.com/translate/docs/quickstarts

In [ ]:
# Imports the Google Cloud Translation library
from google.cloud import translate
from google.cloud import storage

service_account.json 생성법  
https://cloud.google.com/docs/authentication/production?hl=ko

In [ ]:
# 코드를 사용하여 사용자 인증 정보 전달
account_json = "mwp-korean-34ea94931579.json"

def explicit(account_json='service_account.json'):
    # Explicitly use service account credentials by specifying the private key
    # file.
    storage_client = storage.Client.from_service_account_json(account_json)

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)

explicit(account_json)
print(account_json)

[]
mwp-korean-34ea94931579.json


In [ ]:
!export GOOGLE_APPLICATION_CREDENTIALS=account_json
!echo $GOOGLE_APPLICATION_CREDENTIALS

In [ ]:
from google.cloud import translate_v2
from google.cloud.bigquery.client import Client

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/drive/MyDrive/Translate_Google/mwp-korean-34ea94931579.json'
client = translate_v2.Client()
result = client.translate('안녕하세요', target_language='en')
print(result['translatedText'])

Hello


In [ ]:
before_idx = -1

In [ ]:
with open("train_태우_.json", encoding='UTF8') as json_file:
    # APE210K 기준 attr입니다. 다른 attr이면 수정바람.
    question = []
    question_trans = []
    options = []
    correct = []

    for idx, line in tqdm(enumerate(json_file)):

        """if idx > 10:
            break"""

        # translate question
        j = json.loads(line)
        text_question = j['question']
        trans_q = client.translate(text_question, target_language="ko")

        # save info
        question.append(text_question)
        question_trans.append(trans_q['translatedText'])
        options.append(j['options'])
        correct.append(j['correct'])

    frame = pd.DataFrame({'question' : question, 'question_trans' : question_trans, 'options' : options, 'correct' : correct})
    frame.to_csv(str(idx) + "_translate.csv",encoding = 'utf-8')

24369it [33:38, 12.08it/s]


In [ ]:
with open("asdiv-a/validset.json", encoding='UTF8') as json_file:
    # asdiv-a 기준 attr입니다. 다른 attr이면 수정바람.
    id = []
    grade = []
    source = []
    body = []
    body_trans = []
    question = []
    question_trans = []
    solution_type = []
    answer = []
    formula = []

    idx = 0
    for line in tqdm(json.loads(json_file.read())):

        """if idx > 10:
            break"""

        # translate body
        j = line
        text_body = j['Body']
        trans_b = client.translate(text_body, target_language="ko")

        # translate question
        text_question = j['Question']
        trans_q = client.translate(text_question, target_language="ko")

        # save info
        id.append(j['@ID'])
        grade.append(j['@Grade'])
        source.append(j['@Source'])
        body.append(text_body)
        body_trans.append(trans_b['translatedText'])
        question.append(text_question)
        question_trans.append(trans_q['translatedText'])
        solution_type.append(j['Solution-Type'])
        answer.append(j['Answer'])
        formula.append(j['Formula'])

        idx -=- 1

    frame = pd.DataFrame({'@ID' : id, '@Grade' : grade, '@Source' : source,
                          'Body' : body, 'Body_trans' : body_trans,
                          'Question' : question, 'Question_trans' : question_trans,
                          'Solution-Type' : solution_type, 'Answer' : answer, 'Formula' : formula})
    frame.to_csv(str(idx) + "_translate_.csv",encoding = 'utf-8')

100%|██████████| 122/122 [00:20<00:00,  6.06it/s]
